In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# 数据读取
df_train = pd.read_csv('训练集.csv')  # 训练集文件名
df_val = pd.read_csv('验证集.csv')   # 验证集文件名

# 提取训练集和验证集的输入和目标值
X_train = df_train[['longitude', 'latitude']].values  # 训练集输入
y_train = df_train['value'].values  # 训练集目标值

X_val = df_val[['longitude', 'latitude']].values  # 验证集输入
y_val = df_val['value'].values  # 验证集目标值

# 定义距离加权反比法插值函数
def idw_interpolation(x_train, y_train, x_pred, power=2):
    """
    距离加权反比法插值
    :param x_train: 训练点的坐标 (N, 2)
    :param y_train: 训练点的值 (N,)
    :param x_pred: 待预测点的坐标 (M, 2)
    :param power: 距离权重的指数
    :return: 插值结果 (M,)
    """
    predictions = []
    for x in x_pred:
        # 计算每个训练点到当前预测点的距离
        distances = np.linalg.norm(x_train - x, axis=1)
        # 避免除以零的情况，将距离中非常小的值替换为一个较小的正数
        distances = np.where(distances == 0, 1e-10, distances)
        # 计算权重
        weights = 1 / (distances ** power)
        # 插值值计算
        interpolated_value = np.sum(weights * y_train) / np.sum(weights)
        predictions.append(interpolated_value)
    return np.array(predictions)

# 使用距离加权反比法进行插值
y_pred_val = idw_interpolation(X_train, y_train, X_val, power=2)  # power=2 表示平方反比权重

# 计算模型评估指标
mse = mean_squared_error(y_val, y_pred_val)
r2 = r2_score(y_val, y_pred_val)

print("训练和验证完成")
print(f"均方误差 (MSE): {mse:.4f}")
print(f"决定系数 (R²): {r2:.4f}")

# 保存预测结果
df_result = pd.DataFrame({
    'longitude': X_val[:, 0],
    'latitude': X_val[:, 1],
    'actual_value': y_val,
    'predicted_value': y_pred_val
})

df_result.to_csv('predictions-idw.csv', index=False)
print("结果已保存至 'predictions-idw.csv'")


训练和验证完成
均方误差 (MSE): 177.3083
决定系数 (R²): 0.7269
结果已保存至 'predictions-idw.csv'
